# Trabalho Pŕatico 1 - Computação Gráfica
Feito por Gustavo Leão e Luigi Eliabe

## Preparação

Antes de executar o código é necessário preparar o ambiente de execução, para isso é necessário a instalação do [Qt6.6](https://doc.qt.io/qtforpython-6/quickstart.html#quick-start "Site oficial do Qt") para python, assim como o interpretador de [python](https://www.python.org/downloads/ "Site oficial do Python") que seja maior que a 3.7. Após a instalação de toda as dependencias basta executar todas as células.

## Classes

Aqui serão definidas as classes que seram utilizadas no trabalho, elas represetarão os objetos necessário para a criação da janela, como o ponto, a reta, o polígono, a viewport e window do mundo real.

In [ ]:
class Ponto:
    def __init__(self,x,y) -> None:
        self.x = x
        self.y = y
        pass

    def __str__(self) -> str:
        return (f"({self.x},{self.y})")


In [ ]:
class Reta:
    def __init__(self,*pontos) -> None:
        self.pontosReta = []
        for ponto in pontos:
            self.pontosReta.append(ponto)
        pass

    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosReta)+')'

In [ ]:
class Poligono:

    def __init__(self,*pontos) -> None:
        self.pontosPoligono = []
        for ponto in pontos:
            self.pontosPoligono.append(ponto)
        pass

    
    def __str__(self) -> str:
        return '('+','.join(str(e) for e in self.pontosPoligono)+')'

In [ ]:
class Window:
    def __init__(self,minX,minY,maxX,maxY,borda) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.pontos = []
        self.pontos.append(Ponto(self.minX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.maxY))
        self.pontos.append(Ponto(self.minX,self.maxY))

        self.bordas = []
        self.bordas.append(Ponto(self.minX-borda,self.minY-borda))
        self.bordas.append(Ponto(self.maxX+borda,self.minY-borda))
        self.bordas.append(Ponto(self.maxX+borda,self.maxY+borda))
        self.bordas.append(Ponto(self.minX-borda,self.maxY+borda))
        pass

    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

In [ ]:
class Viewport:
    def __init__(self,minX,minY,maxX,maxY) -> None:
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.pontos = []
        self.pontos.append(Ponto(self.minX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.minY))
        self.pontos.append(Ponto(self.maxX,self.maxY))
        self.pontos.append(Ponto(self.minX,self.maxY))
        pass


    def __str__(self) -> str:
        
        return (f"({self.minX},{self.minY},{self.maxX},{self.maxY})")

## Leitura do Arquivo XML

Temos em seguida, a classe responsável por fazer a leitura do arquivo de entrada, nela utilizamos a biblioteca ElementTree para poder fazer a identificação dos objetos e as leituras dos atributos contido nas tags do arquivo. Dentro dessa classe é possível ver comentários adicionados por nós, para servir de auxílio sobre os métodos contidos na biblioteca utilizada, e como foi feita as leituras dos valores.

In [ ]:
class LerXML:
    import xml.etree.ElementTree as Et
    
    def __init__(self,file) -> None:
        
        tree = self.Et.parse(file) #Gera uma árvore com o arquivo
        self.raiz = tree.getroot() #Define a raiz
        borda = 10                 #Borda da Window

        #Percorre os nós do arquivo, procurando todas as TAGS com esse nome
        #Nesse caso estamos percorrendo para pegar os valores da ViewPort
        for view in self.raiz.findall('viewport/vpmin'):
            vpmin = view.attrib #Atribui os atributos da TAG

        for view in self.raiz.findall('viewport/vpmax'):
            vpmax = view.attrib

        vpminX = float(vpmin.get('x')) #Pega o atributo X
        vpminY = float(vpmin.get('y')) #Pega o atributo Y
        vpmaxX = float(vpmax.get('x'))
        vpmaxY = float(vpmax.get('y'))


        #Vamos pegar agora os valores da Window
        for win in self.raiz.findall('window/wmin'):
            wmin = win.attrib

        for win in self.raiz.findall('window/wmax'):
            wmax = win.attrib

        wminX = float(wmin.get('x'))
        wminY = float(wmin.get('y'))
        wmaxX = float(wmax.get('x'))
        wmaxY = float(wmax.get('y'))


        self.janela = Window(wminX,wminY,wmaxX,wmaxY,borda)                  #Criando a instância da Window
        self.viewport = Viewport(vpminX,vpminY,vpmaxX,vpmaxY)    #Criando a instância da Viewport


        #Pegando os valores dos pontos
        self.pontos = [] #Criando um vetor de Pontos
        for pon in self.raiz.findall('ponto'):
            ponto = pon.attrib
            ponX = float(ponto.get('x'))
            ponY = float(ponto.get('y'))
            self.pontos.append(Ponto(ponX,ponY))    #Adicionando os valores ao ponto

        #Pegando valores das Retas
        pontoRetas = [] #Criamos um vetor de pontos da reta
        self.retas = [] #Criamos um vetor com as retas

        for ret in self.raiz.findall('reta'):
            pontoRetas.clear()
            for ponto in ret:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontoRetas.append(Ponto(x,y)) #salvando os pontos da reta em um vetor
            reta = Reta(*pontoRetas)
            self.retas.append(reta) #salvando a reta em um vetor

        
        #Pegar os pontos do Poligono
        pontosPoligono = [] #Fazemos o mesmo esquema das retas
        self.poligonos = []
        for poli in self.raiz.findall('poligono'):
            pontosPoligono.clear()
            for ponto in poli:
                x = float(ponto.attrib.get('x'))
                y = float(ponto.attrib.get('y'))
                pontosPoligono.append(Ponto(x,y))
            poligono = Poligono(*pontosPoligono)
            self.poligonos.append(poligono) #Mandamos um vetor de pontos para salvar o poli
    

## Classe para Criar o Arquivo

Após ser feito a leitura do arquivo e pego os valores contido no mesmo, usamos a seguinte classe para criar um arquivo XML com os valores alterados pela transformação de ViewPort. Nela é importada a mesma biblioteca utilizada para ler o arquivo, para poder manipular e alterar os valores dos atributos das TAGS.

In [ ]:
class CriarXML:
    import xml.etree.ElementTree as Et

    def __init__(self,file,pontos,retas,poligonos) -> None:
        arq = self.Et.parse(file) 
        self.raiz = arq.getroot()

        i=0
        for self.ponto in self.raiz.findall('ponto'):
            self.ponto.set('x',str(pontos[i].x))    #Comando para atualizar os atributos 
            self.ponto.set('y',str(pontos[i].y))    #(Atributo, valor modificar)
            i+=1       #Conta a quantidade de pontos
        i=0
        for self.reta in self.raiz.findall('reta'):
            j=0         #reseta a cada reta, pois uma reta vai ser limitada a 2 pontos
            for h in self.reta.findall('ponto'):
                h.set('x',str(retas[i].pontosReta[j].x))
                h.set('y',str(retas[i].pontosReta[j].y))
                j+=1    #Conta a quantidade de pontos
            i+=1        #Conta a quantidade de retas
            
        i=0
        for self.poli in self.raiz.findall('poligono'):
            j=0         #reseta a cada poligono 
            for h in self.poli.findall('ponto'):
                h.set('x',str(poligonos[i].pontosPoligono[j].x))
                h.set('y',str(poligonos[i].pontosPoligono[j].y))
                j+=1    #Conta a quantidade de pontos no poli
            i+=1        #Conta a quantidade de poli

        arq.write('NovoArquivo.xml')


## Transformada de Viewport

Essa classe pega os valores obtidos no arquivo e transforma para os valores na viewport.

In [ ]:
class Transformada:

    def __init__(self,win,view) -> None:
        self.win = win
        self.view = view
        pass

    def transformarPonto(self,x,y) -> Ponto:
        self.pontoX = (((x-self.win.minX)/(self.win.maxX-self.win.minX))*(self.view.maxX-self.view.minX))
        self.pontoY = ((1-(y-self.win.minY)/(self.win.maxY-self.win.minY))*(self.view.maxY-self.view.minY))
        return Ponto(self.pontoX,self.pontoY)
    
    def transformarReta(self,pontos) -> Reta:
        pontosTrans = []
        for x in pontos:
            pontosTrans.append(self.transformarPonto(x.x,x.y))
        return Reta(*pontosTrans)
            
    
    def transformarPoli(self,pontos) -> Poligono:
        pontosTrans = []
        for x in pontos:
            pontosTrans.append(self.transformarPonto(x.x,x.y))
        return Poligono(*pontosTrans)
            
    def __str__(self) -> str:
        return (f"Viewport: {self.view}\nWindow: {self.win}")

## Aplicação do QT

Agora utilizaremos do Qt para cria uma janela, da qual será possível escolher um arquivo .xml com as medidas da viewport, da window do mundo real, e as coordenadas dentro da window de cada ponto a ser transformado e traçado na viewport.

In [ ]:
from PySide6 import QtCore, QtWidgets, QtGui


In [ ]:
class AddPointDialog(QtWidgets.QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Add Point")
        self.setLayout(QtWidgets.QVBoxLayout())

        # Create widgets
        self.x_input = QtWidgets.QLineEdit()
        self.x_input.setPlaceholderText("Enter X coordinate")
        self.y_input = QtWidgets.QLineEdit()
        self.y_input.setPlaceholderText("Enter Y coordinate")
        self.submit_button = QtWidgets.QPushButton("Add Point")
        self.cancel_button = QtWidgets.QPushButton("Cancel")

        # Layout
        self.layout().addWidget(self.x_input)
        self.layout().addWidget(self.y_input)
        self.layout().addWidget(self.submit_button)
        self.layout().addWidget(self.cancel_button)

        # Connect signals
        self.submit_button.clicked.connect(self.accept)
        self.cancel_button.clicked.connect(self.reject)

    def get_coordinates(self):
        try:
            x = float(self.x_input.text())
            y = float(self.y_input.text())
            return x, y
        except ValueError:
            QtWidgets.QMessageBox.warning(self, "Invalid Input", "Please enter valid numbers for coordinates.")
            return None

In [ ]:
class AddLineDialog(QtWidgets.QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Add Line")
        self.setLayout(QtWidgets.QVBoxLayout())

        # Create widgets
        self.x1_input = QtWidgets.QLineEdit()
        self.x1_input.setPlaceholderText("Enter X1 coordinate")
        self.y1_input = QtWidgets.QLineEdit()
        self.y1_input.setPlaceholderText("Enter Y1 coordinate")
        self.x2_input = QtWidgets.QLineEdit()
        self.x2_input.setPlaceholderText("Enter X2 coordinate")
        self.y2_input = QtWidgets.QLineEdit()
        self.y2_input.setPlaceholderText("Enter Y2 coordinate")
        self.submit_button = QtWidgets.QPushButton("Add Line")
        self.cancel_button = QtWidgets.QPushButton("Cancel")

        # Layout
        self.layout().addWidget(self.x1_input)
        self.layout().addWidget(self.y1_input)
        self.layout().addWidget(self.x2_input)
        self.layout().addWidget(self.y2_input)
        self.layout().addWidget(self.submit_button)
        self.layout().addWidget(self.cancel_button)

        # Connect signals
        self.submit_button.clicked.connect(self.accept)
        self.cancel_button.clicked.connect(self.reject)

    def get_coordinates(self):
        try:
            x1 = float(self.x1_input.text())
            y1 = float(self.y1_input.text())
            x2 = float(self.x2_input.text())
            y2 = float(self.y2_input.text())
            return x1, y1, x2, y2
        except ValueError:
            QtWidgets.QMessageBox.warning(self, "Invalid Input", "Please enter valid numbers for coordinates.")
            return None


In [ ]:
class AddPolygonDialog(QtWidgets.QDialog):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Add Polygon")
        self.setLayout(QtWidgets.QVBoxLayout())

        # Create widgets
        self.coordinates_input = QtWidgets.QLineEdit()
        self.coordinates_input.setPlaceholderText("Enter coordinates as X1,Y1;X2,Y2;... (separated by semicolons)")
        self.submit_button = QtWidgets.QPushButton("Add Polygon")
        self.cancel_button = QtWidgets.QPushButton("Cancel")

        # Layout
        self.layout().addWidget(self.coordinates_input)
        self.layout().addWidget(self.submit_button)
        self.layout().addWidget(self.cancel_button)

        # Connect signals
        self.submit_button.clicked.connect(self.accept)
        self.cancel_button.clicked.connect(self.reject)

    def get_coordinates(self):
        coordinates_text = self.coordinates_input.text()
        points = []
        try:
            for point_text in coordinates_text.split(';'):
                x_text, y_text = point_text.split(',')
                x = float(x_text)
                y = float(y_text)
                points.append(QtCore.QPointF(x, y))
            return points
        except ValueError:
            QtWidgets.QMessageBox.warning(self, "Invalid Input", "Please enter valid coordinates in the format X1,Y1;X2,Y2;...")
            return None


In [ ]:
class ScalingDialog(QtWidgets.QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Scale Elements")

        self.layout = QtWidgets.QVBoxLayout(self)

        self.label = QtWidgets.QLabel("Enter scaling factor:")
        self.layout.addWidget(self.label)

        self.scale_input = QtWidgets.QDoubleSpinBox(self)
        self.scale_input.setRange(0.1, 10.0)  # Allow scaling factors between 0.1 and 10
        self.scale_input.setValue(1.0)  # Default value is 1 (no scaling)
        self.layout.addWidget(self.scale_input)

        self.button_box = QtWidgets.QDialogButtonBox(QtWidgets.QDialogButtonBox.Ok | QtWidgets.QDialogButtonBox.Cancel)
        self.button_box.accepted.connect(self.accept)
        self.button_box.rejected.connect(self.reject)
        self.layout.addWidget(self.button_box)

    def get_scale_factor(self):
        return self.scale_input.value()


In [ ]:
class MyWidget(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()

        # Initialize the scene and view
        self.cena = QtWidgets.QGraphicsScene()
        self.view = QtWidgets.QGraphicsView(self.cena)

        # Layout for the widget
        self.layout = QtWidgets.QVBoxLayout(self)
        self.layout.addWidget(self.view)

        # Create a menu bar
        self.menu_bar = QtWidgets.QMenuBar(self)
        self.layout.setMenuBar(self.menu_bar)

        # Initialize dialogs
        self.add_point_dialog = AddPointDialog()
        self.add_line_dialog = AddLineDialog()
        self.add_polygon_dialog = AddPolygonDialog()
        self.scaling_dialog = ScalingDialog()

        # Create actions
        self.action_open = QtGui.QAction("Open", self)
        self.action_save = QtGui.QAction("Save", self)
        self.action_add_point = QtGui.QAction("Add Point", self)
        self.action_add_line = QtGui.QAction("Add Line", self)
        self.action_add_polygon = QtGui.QAction("Add Polygon", self)
        self.action_rotate = QtGui.QAction("Rotate 90 Degrees", self)
        self.action_scale = QtGui.QAction("Scale Elements", self)
        self.action_move = QtGui.QAction("Move Elements", self)

        # Create menus
        self.file_menu = self.menu_bar.addMenu("File")
        self.add_menu = self.menu_bar.addMenu("Add")
        self.transform_menu = self.menu_bar.addMenu("Transform")

        # Add actions to menus
        self.file_menu.addAction(self.action_open)
        self.file_menu.addAction(self.action_save)
        self.add_menu.addAction(self.action_add_point)
        self.add_menu.addAction(self.action_add_line)
        self.add_menu.addAction(self.action_add_polygon)
        self.transform_menu.addAction(self.action_rotate)
        self.transform_menu.addAction(self.action_scale)
        self.transform_menu.addAction(self.action_move)

        # Connect actions to slots
        self.action_open.triggered.connect(self.LerArq)
        self.action_save.triggered.connect(self.SaveArq)
        self.action_rotate.triggered.connect(self.rotate_all_items)
        self.action_scale.triggered.connect(self.show_scaling_dialog)
        self.action_add_point.triggered.connect(self.show_add_point_dialog)
        self.action_add_line.triggered.connect(self.show_add_line_dialog)
        self.action_add_polygon.triggered.connect(self.show_add_polygon_dialog)
        self.action_move.triggered.connect(self.show_move_buttons)

        # Initialize the list to keep track of added items
        self.items = []

        # Create move buttons
        self.move_buttons_widget = QtWidgets.QWidget()
        self.move_buttons_layout = QtWidgets.QGridLayout(self.move_buttons_widget)
        self.create_move_buttons()
        self.move_buttons_widget.hide()  # Hide initially

    def create_move_buttons(self):
        # Create arrow buttons for movement
        self.button_up = QtWidgets.QPushButton("Up")
        self.button_down = QtWidgets.QPushButton("Down")
        self.button_left = QtWidgets.QPushButton("Left")
        self.button_right = QtWidgets.QPushButton("Right")

        # Set fixed size for smaller buttons
        button_size = QtCore.QSize(50, 30)  # Adjust the size as needed
        self.button_up.setFixedSize(button_size)
        self.button_down.setFixedSize(button_size)
        self.button_left.setFixedSize(button_size)
        self.button_right.setFixedSize(button_size)

        # Optionally adjust font size for better fit
        font = self.button_up.font()
        font.setPointSize(10)  # Adjust the font size as needed
        self.button_up.setFont(font)
        self.button_down.setFont(font)
        self.button_left.setFont(font)
        self.button_right.setFont(font)

        # Add buttons to the layout with adjusted spacing
        self.move_buttons_layout.addWidget(self.button_up, 0, 1)
        self.move_buttons_layout.addWidget(self.button_left, 1, 0)
        self.move_buttons_layout.addWidget(self.button_down, 1, 1)
        self.move_buttons_layout.addWidget(self.button_right, 1, 2)

        # Adjust spacing
        self.move_buttons_layout.setHorizontalSpacing(10)  # Space between buttons in the same row
        self.move_buttons_layout.setVerticalSpacing(10)    # Space between buttons in the same column

        # Connect buttons to slots
        self.button_up.clicked.connect(self.move_up)
        self.button_down.clicked.connect(self.move_down)
        self.button_left.clicked.connect(self.move_left)
        self.button_right.clicked.connect(self.move_right)

        # Add the widget to the layout
        self.layout.addWidget(self.move_buttons_widget)

    @QtCore.Slot()
    def show_move_buttons(self):
        if self.move_buttons_widget.isVisible():
            self.move_buttons_widget.hide()
        else:
            self.move_buttons_widget.show()

    @QtCore.Slot()
    def rotate_all_items(self):
        for item in self.items:
            item.setRotation(item.rotation() + 90)  # Rotate by 90 degrees

    @QtCore.Slot()
    def move_left(self):
        self.move_all_items(-10, 0)

    @QtCore.Slot()
    def move_right(self):
        self.move_all_items(10, 0)

    @QtCore.Slot()
    def move_up(self):
        self.move_all_items(0, -10)

    @QtCore.Slot()
    def move_down(self):
        self.move_all_items(0, 10)

    def move_all_items(self, dx, dy):
        for item in self.items:
            item.moveBy(dx, dy)

    @QtCore.Slot()
    def show_scaling_dialog(self):
        if self.scaling_dialog.exec() == QtWidgets.QDialog.Accepted:
            scale_factor = self.scaling_dialog.get_scale_factor()
            self.scale_all_items(scale_factor)

    def scale_all_items(self, scale_factor):
        for item in self.items:
            item.setScale(scale_factor)

    @QtCore.Slot()
    def show_add_point_dialog(self):
        if self.add_point_dialog.exec() == QtWidgets.QDialog.Accepted:
            coords = self.add_point_dialog.get_coordinates()
            if coords:
                x, y = coords
                pen = QtGui.QPen(QtGui.Qt.black)
                point = self.cena.addEllipse(x, y, 5, 5, pen)
                point.setBrush(QtGui.Qt.black)
                self.items.append(point)

    @QtCore.Slot()
    def show_add_line_dialog(self):
        if self.add_line_dialog.exec() == QtWidgets.QDialog.Accepted:
            coords = self.add_line_dialog.get_coordinates()
            if coords:
                x1, y1, x2, y2 = coords
                pen = QtGui.QPen(QtGui.Qt.black)
                line = QtCore.QLineF(QtCore.QPointF(x1, y1), QtCore.QPointF(x2, y2))
                line_item = self.cena.addLine(line, pen)
                self.items.append(line_item)

    @QtCore.Slot()
    def show_add_polygon_dialog(self):
        if self.add_polygon_dialog.exec() == QtWidgets.QDialog.Accepted:
            points = self.add_polygon_dialog.get_coordinates()
            if points:
                pen = QtGui.QPen(QtGui.Qt.black)
                polygon_item = self.cena.addPolygon(QtGui.QPolygonF(points), pen)
                self.items.append(polygon_item)

    @QtCore.Slot()
    def CriarViewport(self):
        pontosView = self.file.viewport.pontos
        view = []
        for x in pontosView:
            y = QtCore.QPointF(x.x, x.y)
            view.append(y)
        self.cena.addPolygon(QtGui.QPolygonF(view))

    @QtCore.Slot()
    def CriarPonto(self):
        pen = QtGui.QPen(QtGui.Qt.black)
        for x in self.pontosTransformados:
            point = self.cena.addEllipse(x.x, x.y, 2, 2, pen)
            point.setBrush(QtGui.Qt.black)
            self.items.append(point)

    @QtCore.Slot()
    def CriarReta(self):
        for x in self.retasTransformados:
            p1 = x.pontosReta[0]
            p2 = x.pontosReta[1]
            y = QtCore.QLineF(QtCore.QPointF(p1.x, p1.y), QtCore.QPointF(p2.x, p2.y))
            line_item = self.cena.addLine(y)
            self.items.append(line_item)

    @QtCore.Slot()
    def CriarPoli(self):
        view = []
        for x in self.poligonosTransformados:
            for y in x.pontosPoligono:
                view.append(QtCore.QPointF(y.x, y.y))
        polygon_item = self.cena.addPolygon(QtGui.QPolygonF(view))
        self.items.append(polygon_item)

    @QtCore.Slot()
    def LerArq(self):
        self.arq = QtWidgets.QFileDialog.getOpenFileName()[0]
        self.file = LerXML(self.arq)
        trans = Transformada(self.file.janela, self.file.viewport)
        print(trans)
        self.pontosTransformados = []
        self.retasTransformados = []
        self.poligonosTransformados = []
        for x in self.file.pontos:
            self.pontosTransformados.append(trans.transformarPonto(x.x, x.y))
        for retas in self.file.retas:
            self.retasTransformados.append(trans.transformarReta(retas.pontosReta))
        for poli in self.file.poligonos:
            self.poligonosTransformados.append(trans.transformarPoli(poli.pontosPoligono))
        self.CriarViewport()
        self.CriarPonto()
        self.CriarReta()
        self.CriarPoli()

    @QtCore.Slot()
    def SaveArq(self):
        CriarXML(self.arq, self.pontosTransformados, self.retasTransformados, self.poligonosTransformados)


## Main

Essa sessão viabiliza a comunicação do código com o sistema operacional, gerando a interface gráfica.

In [ ]:
import sys
if __name__ == '__main__':
    
    app = QtWidgets.QApplication([])

    widget = MyWidget()
    widget.resize(800, 600)
    widget.show()

    sys.exit(app.exec())

## Observação

Caso a IDE de Jupyter notebook utilizada seja o VSCode, para rodar o código novamente é necessário reiniciar o kernel de execução do jupyter. Isso pode ser feito clicando no botão reiniciar no topo da janela.